<h3> 1.1 Importation des données</h3>

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import scale
import seaborn as sns

In [12]:
# Import as a panda dataframe
dataframe=pd.read_csv("public_dat/store_test.data", sep=' ')

# Drop entries with missing values & first columd
dataframe = dataframe.dropna()

# Display data
dataframe.head()

,store_ID,Day_of_week,Date,Amount_customers,Store_open,Promo,Student_holiday,School_holiday
0,249,5,2014-12-19,725,1,1,0,0
1,190,4,2013-02-28,564,1,0,0,0
2,850,3,2013-06-05,644,1,1,0,0
3,776,6,2015-03-28,435,1,0,0,0
4,14,4,2014-10-02,799,1,1,0,0
